## Практическая работа по лекции 3.
11.03.2024 
БИ-4-23-01
Буров С.А.

`При работе потребуется библиотека TensorFlow. Если не получается её установить, и возникают какие-либо проблемы с совместимостью библиотек или ошибки, задания выполняем в Google Colab`

### Задача 1. Линейная регрессия

Задача прогнозирования числовых рядов решается с помощью различных методов, в том числе и с помощью регрессионного анализа.  Для качественного прогноза в различных сферах (валютные курсы, стоимость ценных бумаг, число заболевших при пандемии и т.д.) используют различные варианты уравнений регрессии, используют преобразование Фурье для выявления сезонной составляющей и другие математические методы. При достаточном упорстве аналитика можно было подобрать комбинацию методов, неплохо предсказывающих поведение анализируемого параметра. Но использование в подобных моделях категориальных данных является затрудничтельным, а текстовых— это вообще отдельной сложная тема.

Решение задачи регрессии с помощью нейронных сетей — иной подход. Рассмотрим его подробнее для начала на примере определения стоимости недвижимости из набора данных `Boston Housing`, включенном в библиотеку `Keras`. Это 13 столбцов различных параметров описывающих недвижимость и всего-то 400 записей с информацией, что является отноисительно небольшим набором данных.

Описание датасета можно посмотреть по [здесь](https://neerc.ifmo.ru/wiki/index.php?title=%D0%98%D0%B7%D0%B2%D0%B5%D1%81%D1%82%D0%BD%D1%8B%D0%B5_%D0%BD%D0%B0%D0%B1%D0%BE%D1%80%D1%8B_%D0%B4%D0%B0%D0%BD%D0%BD%D1%8B%D1%85#Boston_Housing)

    Переменные по порядку:
 `CRIM` Уровень преступности на душу населения по городам
 `ZN` доля жилых земель, зонированных для участков площадью более 25 000 кв. футов.
 `INDUS` доля площадей под неторговую деятельность на город
 Фиктивная переменная `CHAS` Charles River (= 1, если тракт ограничивает реку; 0 в противном случае)
 Концентрация оксидов азота `NOX` (частей на 10 миллионов)
 Среднее количество комнат в жилом помещении по РМ
 `ВОЗРАСТ` Доля квартир, занимаемых владельцами, построенных до 1940 г.
 `DIS` взвесил расстояния до пяти центров занятости Бостона
 Индекс доступности радиальных магистралей РАД
 `TAX` Ставка налога на имущество на полную стоимость за 10 000 долларов США
 Соотношение учеников и учителей `PTRATIO` по городам
 B 1000(Bk - 0,63)^2, где Bk — доля чернокожих по городам.
 `LSTAT` % более низкий статус населения
 `MEDV` Средняя стоимость домов, занимаемых владельцами, в 1000 долларов США


In [ ]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
boston_dataset = load_boston()
boston = pd.DataFrame(boston_dataset.data, columns=boston_dataset.feature_names)
boston['MEDV'] = boston_dataset.target

In [ ]:
X = pd.DataFrame(np.c_[boston['LSTAT'], boston['RM']], columns=['LSTAT', 'RM'])
Y = boston['MEDV']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=5)
lin_model = LinearRegression()
lin_model.fit(X_train, Y_train)

In [ ]:
y_train_predict = lin_model.predict(X_train)
rmse = (np.sqrt(mean_squared_error(Y_train, y_train_predict)))
print("СКО на тренировочных данных", rmse)
y_test_predict = lin_model.predict(X_test)
rmse = (np.sqrt(mean_squared_error(Y_test, y_test_predict)))
print("СКО на учебных данных", rmse)

### Задача 2. Регрессия полносвязной нейронной сетью

#### 2.1 Подготовка данных

In [ ]:
from tensorflow.keras.datasets import boston_housing #другой источник датасета boston_housing
from tensorflow.keras.models import Sequential #контейнеры, в которых хранятся последовательности
from tensorflow.keras.layers import Dense #для создания слоёв нейронной сети
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline 

from sklearn.datasets import load_boston

In [ ]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data() #загрузка данных

In [ ]:
#функция стандартизации данных
def norm(x):
  return (x - np.mean(x)) / np.std(x)
x_train = norm(x_train); #стандартизация данных
x_test = norm(x_test); #стандартизация данных

На прошлой лекции мы рассматривали общие принципы построения нейронных сетей и говорили о функциях активации нейронов. В нашей сети в качестве функции активации будем использовать сигмоиду. Функция аткивации сигмоиды на выходе имеет значение в диапазоне от 0 до 1. В связи с эитм, необходимо привести значение y_train к диапазону от 0 до 1

In [ ]:
# Приводим значения ответов в диапазон от 0 до 1 (нормализация данных)
min_y = y_train.min()
y_train = y_train - min_y #смещаем ответы к 0, вычитая минимальное значение
max_y = y_train.max() #берем максимум от уже "опущенных" вниз на min_y данных 
y_train /= max_y #приводим к 1 обучающую выборку
#нормируем ответы проверочной выборки, используя статистику min_y и max_y обучающей 
y_test = y_test - min_y
y_test /= max_y

#### 2.2 Построение нейронной сети

Строим простую полносвязную нейронную сеть (feed forward neural network). Выходной слой с одним линейным нейроном — для задачи регрессии. Функция активации — RELU в промежуточном слое и sigmoid в выходном. Конфигурация сети взята из примера — https://www.tensorflow.org/tutorials/keras/basic_regression

In [ ]:
x_train.shape

In [ ]:
model = Sequential()
model.add(Dense(13, activation='relu', input_shape=(x_train.shape[1],))) #13 - количество нейронов
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid')) # sigmoid, т.к. данные от 0 до 1

In [ ]:
#Просмотр архитектуры построенной сети
print(model.summary()) # архитектура нашей модели

In [ ]:
# Т.к. задача регрессии, удобнее использовать mean square error(средне-квадратичная ошибка).
# В качестве метрики берем mean absolute error (средний модуль ошибки)
model.compile(optimizer='rmsprop', loss='mse', metrics=['mae'])

In [ ]:
history = model.fit(x_train, 
                    y_train, 
                    epochs=20, 
                    validation_split=0.1, 
                    verbose=2)

#### 2.3. Проверка работы нейронной сети

In [ ]:
# Делаем прогноз. Возвращается копия предсказания в виде одномерного массива
pred = model.predict(x_test).flatten() 
# Возвращаем к прежнему размеру, так нормализировали значение целевой функции в диапазон от 0 до 1
pred = pred * max_y + min_y 
y_test = y_test * max_y + min_y

In [ ]:
# Предсказание vs правильный ответ
for i in range(len(pred)):
  print("Предсказание: ", round(pred[i],2), ", а верный ответ: ", round(y_test[i],2), ", разница: ", round(pred[i] - y_test[i],2))

Визуализация данных

In [ ]:
# График срдней абсолютной ошибки (MAE)

plt.plot(history.history['mae'], 
         label='Средняя абсолютная ошибка на обучающем наборе')
plt.plot(history.history['val_mae'], 
         label='Средняя абсолютная ошибка на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Средняя абсолютная ошибка')
plt.legend()
plt.show()

In [ ]:
# Диаграмма рассеяния предсказаний
plt.scatter(y_test, pred) 
plt.xlabel('Правильные значение, $1K')
plt.ylabel('Предсказания, $1K')
plt.axis('equal')
plt.xlim(plt.xlim())
plt.ylim(plt.ylim())
plt.plot([-100, 100], [-100, 100])
plt.show()

In [ ]:
#Частотное распределние ошибок (гистограмма ошибок)
error = pred - y_test
#Построение гистограммы
plt.hist(abs(error), bins = 25)
plt.xlabel("Значение ошибки, тыс.$")
plt.ylabel("Количество")
plt.show()

### 2.4 Самостоятельное задание

##### 2.4.1 Постройте нейронную сеть, визуилизируйте результаты её работы аналогично предыдущей сети, согласно следующим параметрам:
    - параметры сети аналогичны предыдущему примеру
    - при обучении использовать метод адптивного градиентного спуска adam

In [ ]:
#ваш код здесь

##### 2.4.2 Постройте нейронную сеть, визуилизируйте результаты её работы аналогично предыдущей сети, согласно следующим параметрам:
    - функция активации неройнов - сигмоида

In [ ]:
#ваш код здесь

##### 4.3 Постройте нейронную сеть, визуализируйте результаты её работы аналогично предыдущей сети, согласно следующим параметрам:
    - число нейронов в промежуточных слоях - 32
    - количество слоёв нейронов на 1 больше

In [ ]:
#ваш код здесь

##### 2.4.3 Постройте нейронную сеть, визуилизируйте результаты её работы аналогично предыдущей сети, согласно вашей подобранной комбинации параметров, которая работалы бы лучше, чем предложенная преподавателем модель.

In [ ]:
#ваш код здесь

Сравните результаты и напишите краткие выводы (какой из вариантов более точен)

Выводы

### Задача 3. Классификация полносвязной нейронной сетью

Будем решать задачу классификации одежды на датасет `Fashion MNIST`. Все
данные (изображения одежды) в `Fashion MNIST` подразделяются на 10 классов
- 0 T-shirt/top (футболка/топ)
- 1 Trouser (брюки)
- 2 Pullover (пуловер)
- 3 Dress (платье)
- 4 Coat (пальто)
- 5 Sandal (сандалии)
- 6 Shirt (рубашка)
- 7 Sneaker (кеды)
- 8 Bag (сумка)
- 9 Ankle boot (ботильоны)

Датасет представляет собой набор изоражений размером 28x28

![Fashion MNIST](./fashion.jpg)

Необходимо построить полносвязную нейронную сеть для классфикации изображений

![NeuroLink](./neurolink.jpg)

 Разобрать самостоятельно пример классификации с помощью нейросети данного датасета [здесь](https://habr.com/ru/post/454034/). Добавить код в блокнот


In [ ]:
#ваш код здесь